<a href="https://colab.research.google.com/github/mansivyas26/perfusion_dataPipeline/blob/main/variables_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import pandas as pd
import numpy as np
import random
import ast
import warnings
warnings.filterwarnings('ignore')

In [138]:
#read individual runs file .pkl
perfusion_data = pd.read_pickle("/content/drive/MyDrive/Steve_BEXT84_01-17-2024_13-56-14_all_data.pkl")

In [139]:
#read variable dictionary
variable_dict = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/test_262 - test.csv")

variable_dict['possible_variables'] = variable_dict['possible_variables'].apply(ast.literal_eval)
#variable_dict.to_csv("/content/drive/MyDrive/Pefusion_DataPipeline/test.csv")


In [140]:
#reading the existing schema
schema = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/m12_full_join.csv",nrows=10)

In [141]:
schema_cols = schema.columns
l = schema_cols.to_list()

In [51]:
for val in l:
  if val == "glucose_pump_rate_setpoint":
    print(val)


glucose_pump_rate_setpoint


In [142]:
perfusion_data_subset = perfusion_data.head(3)

In [143]:
#check scehma with variable dictionary
def check_schema(schema,variable_dict):
  columns = schema.columns.tolist()
  for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
            if column in row['possible_variables']:
               # print(column)
                match_found = True
                if column != row['standard_variable']:
                    print("columns that were renamed",column)
                    #print(f"Changing column {column} to {row['standard_variable']}")
                    schema.rename(columns={column: row['standard_variable']}, inplace=True)

In [144]:
check_schema(schema,variable_dict)

columns that were renamed glucose_control_mode
columns that were renamed syringe_pump_manual_setrate
columns that were renamed time
columns that were renamed probe_ph_mv_measured
columns that were renamed glucose_pump_rate_setpoint


In [145]:
#code to check schema and pkl files with variable dict
def check_runs_with_variable_dict_new(pkl_df, variable_dict, schema_subset):

    removed_columns = []  # List to keep track of REMOVED columns
    list_of_files_ignored_in_pkl = ['timestamp','glucose_syringe_pump_buzzer']
    list_of_files_ignored_in_schema = ['resistance_qc_solenoid_flag','perfusion_date','perfusion_id','control_time','glucose_syringe_pump_dispensed_control', 'glucose_syringe_pump_buzzer_control']

    temporary_none = ['dialysis_compensate_scale_weight','dialysis_flux_valve_position','viscometer_temperature','dialysis_post_pressure','dialysis_compensate_pump_rate','dialysis_compensate_scale_flux','dialysis_pre_pressure','dialysis_pre_pressure_setpoint','dialysis_pressure_valve_position','dialysis_waste_scale_flux','dialysis_waste_scale_weight','']
    for idx, row in variable_dict.iterrows():
            if row['standard_variable'] in list_of_files_ignored_in_schema:
                  pkl_df[row['standard_variable']] = "to be added"
    columns = pkl_df.columns.tolist()
    for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
           if column in row['possible_variables']:
                match_found = True
                if row['variable_status'] == "EXISTING" and column != row['standard_variable']:
                    #print(f"Changing column {column} to {row['standard_variable']}")
                    pkl_df.rename(columns={column: row['standard_variable']}, inplace=True)
                    if column in schema_subset.columns:
                        schema_subset.rename(columns={column: row['standard_variable']}, inplace=True)
                elif row['variable_status'] == "NEW":
                   # print(f"Column status changes to existing. New column to be added to schema: {row['standard_variable']}")
                    #add a column with name row['standard_variable'] and  value = None
                    variable_dict.loc[idx,'variable_status'] = "EXISTING"

                    if row['standard_variable'] not in schema_subset.columns:
                        schema_subset[row['standard_variable']] = None

                if row['variable_status'] == "REMOVED":
                   # print(f"Removed column identified: {column}")
                    removed_columns.append(column)
                    for c in removed_columns:
                      print(removed_columns)
                      pkl_df[c] = None

                break
        for val in temporary_none:
          if val in schema_subset.columns and val not in pkl_df.columns:
        # Add a new column to the pkl DataFrame with the value initialized as None
            pkl_df[val] = None

        if not match_found:
          if row['variable_status'] == "REMOVED" or column == "None":
            pkl_df[row['standard_variable']] = "removed"

       # pkl_df = pkl_df.drop(columns=list_of_files_ignored_in_pkl,inplace=True)








In [149]:
def check_columns_match(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    if columns_df1 != columns_df2:
        columns_only_in_df1 = columns_df1 - columns_df2
        columns_only_in_df2 = columns_df2 - columns_df1

        print("Columns only in .pkl file:", columns_only_in_df1)
        print("Columns only in schema:", columns_only_in_df2)
        print(len(columns_only_in_df2),"only_schema")
        print(len(columns_only_in_df1),"only_pkl")

        return False
    else:
        print(len(df1.columns))
        print(len(df2.columns))
        print("Columns match between the two DataFrames.")
        return True

def append_df_row_wise(pkl_df, schema_df):
    #pkl_df = pkl_df.drop(columns=['None'],inplace=True)

    # Check if columns match
    if not check_columns_match(pkl_df, schema_df):
      return None
    else:
      schema_df = schema_df.sort_index()
      schema_df = schema_df[sorted(schema_df.columns)]
      pkl_df = pkl_df[sorted(pkl_df.columns)]
      #print(pkl_df)

      """schema_df.reset_index(drop=True, inplace=True)
      schema_duplicates = schema_df.index.duplicated()
      print("schema",schema_df[schema_duplicates])
      pkl_df.reset_index(drop=True,inplace=True)"""

      #combined_df = schema_df.append(pkl_df,ignore_index=False)
      #print(combined_df)


In [152]:
print(append_df_row_wise(perfusion_data_subset,schema))

Columns only in .pkl file: {'glucose_syringe_pump_buzzer', 'timestamp', 'None'}
Columns only in schema: set()
0 only_schema
3 only_pkl
None


In [151]:
print(check_runs_with_variable_dict_new(perfusion_data_subset,variable_dict,schema))

['control_time']
['control_time', 'glucose_syringe_pump_buzzer_control']
['control_time', 'glucose_syringe_pump_buzzer_control']
None


In [ ]:
list_of_files_ignored_in_pkl = ['timestamp','glucose_syringe_pump_buzzer',]

In [136]:
perfusion_data_subset

,time(in hrs),sensorbox_drain_solenoid_1_flag,auto_sampler_flag,exit_all_flag,oxyline_flow_measured,brain_flow_target,raman_glucose_mg_dl_measured,heat_exchanger_temperature_setpoint,gas_mixer_status_flag,mettler_ph_act,...,brain_flow_range_calculated,brain_flow_range_calculated,brain_flow_range_calculated,perfusion_date,perfusion_id,resistance_qc_solenoid_flag,control_time,brain_flow_range_calculated,glucose_syringe_pump_buzzer_control,glucose_syringe_pump_dispensed_control
0,0.000000,NaN,NaN,NaN,-3.614960,NaN,NaN,NaN,NaN,NaN,...,None,removed,removed,to be added,to be added,to be added,None,removed,None,to be added
1,0.000278,1.0,-1.0,0.0,-4.802561,0.0,NaN,20.0,NaN,NaN,...,None,removed,removed,to be added,to be added,to be added,None,removed,None,to be added
2,0.000556,1.0,1.0,0.0,-2.030541,0.0,NaN,20.0,NaN,NaN,...,None,removed,removed,to be added,to be added,to be added,None,removed,None,to be added


In [125]:
p_filter = perfusion_data_subset['brain_flow_range_calculated']

In [126]:
p_filter

,brain_flow_range_calculated,brain_flow_range_calculated,brain_flow_range_calculated,brain_flow_range_calculated,brain_flow_range_calculated
0,NaN,None,removed,removed,removed
1,0.0,None,removed,removed,removed
2,0.0,None,removed,removed,removed
